In [1]:
ref = require('./module');
for (let key in ref) {
  global[key] = ref[key];
}

[Function: lookupBST]

In [11]:
function test(name, expect, testcase)
{
    var real = testcase;
    if (real != expect)
    {
        throw "Test " + name + " failed: " + String(real) + " : Expected " + String(expect);
    }
    else
    {
        console.log("Pass: " + name);
    }
}

In [2]:
/* Page 5: The type tag will make it recognizable as a variable. Now
you may finish the definitions of the type predicate and visitors of
variable. */
function variable(name)
{
    return pair("variable", pair(name, null))
}

In [3]:
/* Exercise set 5 */

// The isVariable function is used to check if the input x is a variable
// Input: x - object to be checked
function isVariable(x)
{
    if (!isPair(x))
    {
        return false;
    }
    else
    {
        return first(x) == "variable";
    }
}

// The variableName function is used to get the name of a variable
// Input: v - variable object
function variableName(v)
{
    return first(second(v));
}

// The fun function is used to create a function with specific parameters and function body
// Input: param - function parameter, body - function body
function fun(param, body)
{
    return pair("function", pair(param, body));
}

// The isFunction function is used to check if the input x is a function
// Input: x - object to be checked
function isFunction(x)
{
    if (!isPair(x))
    {
        return false;
    }
    else
    {
        return first(x) == "function";
    }
}

// The funParam function is used to get the parameter of a function
// Input: f - function object
function funParam(f)
{
    return first(second(f));
}

// The funBody function is used to get the content of a function
// Input: f - function object
function funBody(f)
{
    return second(second(f));
}

// The closure function is used to create a closure
// Input: fun - function, env - closure environment
function closure(fun, env)
{
    return pair("closure", pair(fun, env));
}

// The isClosure function is used to check if the input x is a closure
// Input: x - object to be checked
function isClosure(x)
{
    if (!isPair(x))
    {
        return false;
    }
    else
    {
        return head(x) == "closure";
    }
}

// The closureFun function is used to get the function part of a closure
// Input: c - closure object
function closureFun(c)
{
    return first(second(c));
}

// The closureEnv function is used to get the environment part of a closure
// Input: c - closure object
function closureEnv(c)
{
    return second(second(c));
}

// The call function is used to create a call
// Input: op - operation (which could be function expressions, variables that point to the function, or another call that calls the returning function), arg - parameter for the operation
function call(op, arg)
{
    return pair("call", pair(op, arg));
}

// The isCall function is used to check if the input x is a call
// Input: x - object to be checked
function isCall(x)
{
    if (!isPair(x))
    {
        return false;
    }
    else
    {
        return head(x) == "call";
    }
}

// The callOp function is used to get the operation part of a call (operator)
// Input: c - call object
function callOp(c)
{
    return first(second(c));
}

// The callArg function is used to get the parameter part of a call (argument or operand)
// Input: c - call object
function callArg(c)
{
    return second(second(c));
}

In [4]:
/* Page 6: Can you translate the JavaScript function
x => y => x + y into our language? */
var func = fun("x", fun("y", binop("+", variable("x"), variable("y"))));

In [9]:
/* Page 6: Now translate the call (x => y => x + y)(2)(3) into our language.*/
var callfunc = call(call(func, 2), 3);

In [6]:
/* Page 6: we need multiple branches in the interpreter. Its structure
looks like this. */
var lookupEnv = lookupTable;
var extEnv = addTable;
function interp(exp, env)
{
    if (typeof(exp) == "number")
    {
        return exp;
    }
    else if (isVariable(exp))
    {
        var value = lookupEnv(variableName(exp), env);
        if (value == null)
        {
            throw "Undefined variable: " + variableName(env);
        }
        else
        {
            return value;
        }
    }
    else if (isFunction(exp))
    {
        return closureEnv(exp, env);
    }
    else if (isCall(exp))
    {
        var op = interp(callOp(exp), env);
        var arg = interp(callArg(exp), env);
        if (!isClosure(op))
        {
            throw "Illegal call operator: " + pairToString(op);
        }
        else
        {
            var f = closureFun(op);
            var newEnv = extEnv(funParam(f), arg, closureEnv(op));
            return interp(funBody(f), newEnv);
        }
    }
    else if (isBinOp(exp))
    {
        var op = binopOp(exp);
        var e1 = binopE1(exp);
        var e2 = binopE2(exp);
        if (op == "+")
        {
            return interp(e1, env) + interp(e2, env);
        }
        else if (op == "-")
        {
            return interp(e1, env) - interp(e2, env);
        }
        else if (op == "*")
        {
            return interp(e1, env) * interp(e2, env);
        }
        else if (op == "/")
        {
            return interp(e1, env) / interp(e2, env);
        }
        else
        {
            throw "Illegal binop operator: " + op;
        }
    }
    else
    {
        throw "Illegal expression: " + pairToString(exp);
    }
}

var s1 = binop("*", 2, 3);
var s2 = binop("+", 1, binop("*", 2, 3));
console.log(interp(s1))
console.log(interp(s2))

6
7


In [ ]:
// --------- Exercise Set 7 ---------

In [10]:
// var lookupEnv = lookupTable;
// var extEnv = addTable;

var lookupEnv = lookupBST;
var extEnv = addBST;

In [31]:
// --------- if ---------
function ifExp(condition, trueBranch, falseBranch) {
  return pair("ifExp", pair(condition, pair(trueBranch, pair(falseBranch, null))));
}

function isIfExp(x) {
  return isPair(x) && first(x) == "ifExp";
}

function ifCondition(ifexp) {
  return first(second(ifexp));
}

function ifTrueBranch(ifexp) {
  return first(second(second(ifexp)));
}

function ifFalseBranch(ifexp) {
  return first(second(second(second(ifexp))));
}

In [49]:
// let function
function letExp(name, value, body)
{
    return pair("let", pair(name, pair(value, pair(body, null))));
}

function isLet(exp)
{
    return first(exp) == "let";
}

function letName(exp)
{
    return first(second(exp));
}

function letValue(exp)
{
    return first(second(second(exp)));
}

function letBody(exp)
{
    return first(second(second(second(exp))));
}

In [68]:
function expToString(exp) {
  if (typeof(exp) == "number" || 
      typeof(exp) == "string" ||
      typeof(exp) == "boolean") {
    return exp;
  } else if (isVariable(exp)) {
    return variableName(exp);
  } else if (isFunction(exp)) {
    return funParam(exp) +
        " => " +
        expToString(funBody(exp));
  } else if (isCall(exp)) {
      var op = callOp(exp)
      if (isFunction(op))
      {
          return "(" +
                expToString(callOp(exp)) +
                ")(" +
                expToString(callArg(exp)) +
                ")";
      }
      else
      {
          return expToString(callOp(exp)) +
                "(" +
                expToString(callArg(exp)) +
                ")";
      }
  } else if (isBinOp(exp)) {
    return "(" + 
        expToString(binopE1(exp)) +
        " " +
        binopOp(exp) +
        " " +
        expToString(binopE2(exp)) +
        ")";
  } else if (isClosure(exp)) {
    // Closures don't have a standard display mode. To aid understanding,
    // we'll design our own display mode.
    // The display content is the function plus the content of env,
    // with a colon in between them.
    // The function needs recursive call to expToString, env can call listToString.
    return expToString(closureFun(exp)) + " : " + listToString(closureEnv(exp));
  } else if (isIfExp(exp)) {
      return "if (" +
          expToString(ifCondition(exp)) +
          ")\n{\n\treturn " +
          expToString(ifTrueBranch(exp)) +
          ";\n}\nelse\n{\n\treturn " +
          expToString(ifFalseBranch(exp)) +
          ";\n}"
  } else if (isLet(exp)) {
      return "let((" + expToString(letName(exp)) + " " +
          expToString(letValue(exp)) + ") " + expToString(letBody(exp)) + ")";
  }else {
    // An error message for illegal expressions.
    throw "illegal expression";
  }
}

// // Tests
test("test expToString variable", "x", expToString(variable("x")));

test(
  "test expToString function",
  "x => x",
  expToString(fun("x", variable("x")))
);

test(
  "test expToString function with binop 1",
  "x => (x + 2)",
  expToString(fun("x", binop("+", variable("x"), 2)))
);

test(
  "test expToString function with binop 2",
  "x => (1 + (x * 2))",
  expToString(
    fun(
      "x", 
      binop("+", 1, binop("*", variable("x"), 2))
    )
  )
);

test(
  "test expToString call op variable",
  "f(2)",
  expToString(call(variable("f"), 2))
);

test(
  "test expToString call nested",
  "f(g(2))",
  expToString(call(variable("f"), call(variable("g"), 2)))
);

test(
  "test expToString curried call",
  "(x => y => (x + y))(2)(3)",
  expToString(
    call(
      call(
        fun(
          "x",
          fun("y", binop("+", variable("x"), variable("y"))
        )),
        2
      ),
      3
    )
  )
);

Pass: test expToString variable
Pass: test expToString function
Pass: test expToString function with binop 1
Pass: test expToString function with binop 2
Pass: test expToString call op variable
Pass: test expToString call nested
Pass: test expToString curried call


In [74]:
// complete interpreter
function interp(exp, env)
{
    if (typeof(exp) == "number" ||
        typeof(exp) == "string" ||
        typeof(exp) == "boolean"
       )
    {
        return exp;
    }
    else if (isVariable(exp))
    {
        var value = lookupEnv(variableName(exp), env);
        if (value == null)
        {
            throw "Undefined variable: " + variableName(exp);
        }
        else
        {
            return value;
        }
    }
    else if (isFunction(exp))
    {
        return closure(exp, env);
    }
    else if (isCall(exp))
    {
        var op = interp(callOp(exp), env);
        var arg = interp(callArg(exp), env);
        if (!isClosure(op))
        {
            throw "Illegal call operator: " + pairToString(op);
        }
        else
        {
            var f = closureFun(op)
            var newEnv = extEnv(funParam(f), arg, closureEnv(op));
            return interp(funBody(f), newEnv);
        }
    }
    else if (isClosure(exp))
    {
        return exp;
    }
    else if (isBinOp(exp))
    {
        var op = binopOp(exp);
        var e1 = binopE1(exp);
        var e2 = binopE2(exp);
        if (op == "+")
        {
            return interp(e1, env) + interp(e2, env);
        }
        else if (op == "-")
        {
            return interp(e1, env) - interp(e2, env);
        }
        else if (op == "*")
        {
            return interp(e1, env) * interp(e2, env);
        }
        else if (op == "/")
        {
            return interp(e1, env) / interp(e2, env);
        }
        else if (op == "==")
        {
            return interp(e1, env) == interp(e2, env);
        }
        else if (op == ">")
        {
            return interp(e1, env) > interp(e2, env);
        }
        else if (op == "<")
        {
            return interp(e1, env) < interp(e2, env);
        }
        else
        {
            throw "Illegal binop operator: " + op;
        }
    }
    else if (isIfExp(exp))
    {
        var condition = interp(ifCondition(exp), env);
        if (condition)
        {
            return interp(ifTrueBranch(exp), env);
        }
        else
        {
            return interp(ifFalseBranch(exp), env);
        }
    }
    else if (isLet(exp))
    {
        // --------- let type I ---------
        // var newenv = extEnv(letName(exp), interp(letValue(exp)), env);
        // return interp(letBody(exp), newenv);
        // --------- let type II ---------
        var letcall = call(fun(letName(exp), letBody(exp)), 
                           interp(letValue(exp), env));
        return interp(letcall, env);
    }
    else
    {
        throw "Illegal expression: " + pairToString(exp);
    }
}

var show = console.log;
// Testing
show("------ interp ------");
test("test number", 42, interp(42));
test("test string", "good", interp("good"));

// ------- identity function --------
var idFun = fun("x", variable("x"));

// test("test idFun text",
//   "(closure (function x (variable x)) ())",
//   listToString(interp(idFun))
// );

test("test identity function call",
  6,
  interp(call(idFun, 6))
);

// ------- x => x * x --------
var sqFun = fun("x", binop("*", variable("x"), variable("x")));

// test("test x => x * x text",
//   "(closure (function x (binop * (variable x) (variable x))) ())",
//   listToString(interp(sqFun))
// );

test("test x => x * x call",
  9,
  interp(call(sqFun, 3))
);

// Display the internal structure of 'expression' of sqFun
// show(expToString(sqFun)); // x => (x * x)

// Display the internal structure of 'value' (closure) of sqFun
// show(expToString(interp(sqFun))); // x => (x * x) : ()

// -------- curried add --------
// x => y => x + y 
var curriedAdd =  fun("x",
  fun("y", binop("+", variable("x"), variable("y")))
);

test("test curriedAdd",
  5,
  interp(call(call(curriedAdd, 2), 3))
);

// Display the internal structure of the returned closure in curriedAdd(2)

// show(expToString(interp(call(curriedAdd, 2)))); // table: y => (x + y) : ((x . 2))

// --------- apply (curried) ---------
// f => x => f(x)
var applyFun = fun("f",
  fun("x", call(variable("f"), variable("x")))
);

// apply(sqFun)(3)
test("test apply call",
  9,
  interp(call(call(applyFun, sqFun), 3))
);

// // -------- compose (curried) ---------
// // f => g => x => f(g(x))
var composeFun = fun("f",
  fun("g", 
      fun("x", 
          call(variable("f"), call(variable("g"), variable("x")))))
);

var add1 = fun("x", binop("+", variable("x"), 1));
var square = fun("x", binop("*", variable("x"), variable("x")));

// var composed = compose(add1)(square);
var composed1 = call(call(composeFun, add1), square);

// composed(3)
var composeCall1 = call(composed1, 3);
test("test composed call 1", 10, interp(composeCall1));

// compose(square)(add1)(3)
var composeCall2 = call(call(call(composeFun, square), add1), 3);
test("test composed call 2", 16, interp(composeCall2));

// function as param
var funCall = call(fun("f", call(variable("f"), 42)), fun("x", variable("x")));
test("test function as param", 42, interp(funCall));
// ---------- if -----------
test("if constant true", "cat", interp(ifExp(true, "cat", "dog")));

test("if constant false", "dog", interp(ifExp(false, "cat", "dog")));

test("test if binop true", "cat", interp(ifExp(binop("<", 1, 2), "cat", "dog")));

test("test if binop false", "dog", interp(ifExp(binop(">", 1, 2), "cat", "dog")));

test(
  "test if fun true",
  "cat",
  interp(
    call(
      fun("f", ifExp(call(variable("f"), 12), "cat", "dog")),
      fun("x", binop(">", variable("x"), 7))
    )
  )
);

test(
  "test if fun false",
  "dog",
  interp(
    call(
      fun("f", ifExp(call(variable("f"), 5), "cat", "dog")),
      fun("x", binop(">", variable("x"), 7))
    )
  )
);

// abs
var absExp = fun("x",
  ifExp(
    binop("<", variable("x"), 0),
    binop("-", 0, variable("x")),
    variable("x")
  )
);

test("test abs 1", 3, interp(call(absExp, 3)));

test("test abs 2", 7, interp(call(absExp, -7)));

test("test abs 3", 0, interp(call(absExp, 0)));

show("-------- let ---------");
// (let ((x 2))
//    x)
var let1 = letExp("x", 2, variable("x"));
test("test let 1", 2, interp(let1));

// (let ((x (* 2 3)))
//   x)
var let1a = letExp("x",
                   binop("*", 2, 3),
                   variable("x"));

test("test let 2", 6, interp(let1a));

// (let ((x 2))
//    (* x 3))
var let2 = letExp("x", 2, binop("*", variable("x"), 3));
test("test let 3", 6, interp(let2));

// (let ((x 2))
//  (let ((y 3))
//    (+ x y))
var let3 = letExp("x", 2,
                  letExp("y", 3,
                         binop("+", variable("x"), variable("y"))));

test("test nested let", 5, interp(let3));

var let4 = letExp("f", fun("x", variable("x")),
                  call(variable("f"), 42));
                  
test("test let 4",
     42,
     interp(let4));

------ interp ------
Pass: test number
Pass: test string
Pass: test identity function call
Pass: test x => x * x call
Pass: test curriedAdd
Pass: test apply call
Pass: test composed call 1
Pass: test composed call 2
Pass: test function as param
Pass: if constant true
Pass: if constant false
Pass: test if binop true
Pass: test if binop false
Pass: test if fun true
Pass: test if fun false
Pass: test abs 1
Pass: test abs 2
Pass: test abs 3
-------- let ---------
Pass: test let 1
Pass: test let 2
Pass: test let 3
Pass: test nested let
Pass: test let 4


In [36]:
// Y combinator
var Y = f =>
    (x => f(v => x(x)(v)))
    (x => f(v => x(x)(v)));

var factGen = fact =>
    n =>
        {
            if (n == 0)
            {
                return 1;
            }
            else
            {
                return n * fact(n - 1);
            }
        };

var yfact = Y(factGen);
show(yfact(5));

120


In [47]:
var YExpComp = fun("x", 
                   call(variable("f"),
                       fun("v",
                          call(call(variable("x"), variable("x")),
                              variable("v")))));
var YExp = fun("f", call(YExpComp, YExpComp));
var factGenExp = fun("fact",
                    fun("n", 
                        ifExp(binop("==", variable("n"), 0),
                                1,
                                binop("*", variable("n"), 
                                      call(variable("fact"), 
                                           binop("-", variable("n"), 1)))
                       )));//TODO;
    
test("test YExp to string",     
     "f => (x => f(v => x(x)(v)))(x => f(v => x(x)(v)))",     
     expToString(YExp));
console.log(expToString(factGenExp));
var factExp = call(YExp, factGenExp);
test("test Y combinator 1", 120, interp(call(factExp, 5)));

show("all in one fact(5): " +  
     interp(
         call(call(fun("f", call(
             fun("x", 
                   call(variable("f"),
                       fun("v",
                          call(call(variable("x"), variable("x")),
                              variable("v"))))), 
             fun("x", 
                   call(variable("f"),
                       fun("v",
                          call(call(variable("x"), variable("x")),
                              variable("v"))))))),
             fun("fact",
                    fun("n", 
                        ifExp(binop("==", variable("n"), 0),
                                1,
                                binop("*", variable("n"), 
                                      call(variable("fact"), 
                                           binop("-", variable("n"), 1)))
                       )))), 
              5)
         )
    );

Pass: test YExp to string
fact => n => if ((n == 0))
{
	return 1;
}
else
{
	return (n * fact((n - 1)));
}
Pass: test Y combinator 1
all in one fact(5): 120
